In [345]:
import numpy as np 
import h5py
import os 
import random 
import itertools
import math
import sys

import torch 
import torch.nn as nn 
import torch.nn.functional as F 
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import Sampler
from torch.optim import SGD
import torchvision.transforms as transforms
from skimage.measure import label 

import torch.backends.cudnn as cudnn

In [346]:
!pip install medpy

In [347]:
from tensorboardX import SummaryWriter
import shutil
import argparse
import logging
import random
from medpy import metric
import pdb
from tqdm import tqdm

#### 1. BaseDataset

In [349]:
import os
import h5py
from torch.utils.data import Dataset

class LAHeart(Dataset):
    def __init__(self, base_dir, split='train', transform=None, num=None):
        self._base_dir = base_dir
        self.split = split
        self.transform = transform
        self.sample_list = []
        
        # Path for train/test list
        list_file = os.path.join(self._base_dir, f"{split}.list")
        if not os.path.isfile(list_file):
            raise ValueError(f"The {split} list file is missing: {list_file}")
        
        with open(list_file, 'r') as file:
            self.sample_list = [item.strip() for item in file.readlines()]
        
        if num is not None:
            self.sample_list = self.sample_list[:num]

        print(f"Mode = {self.split}, total samples: {len(self.sample_list)}")

    def __len__(self):
        return len(self.sample_list)

    def __getitem__(self, index):
        case = self.sample_list[index]
        file_path = os.path.join(self._base_dir, f'2018LA_Seg_Training Set/{case}/mri_norm2.h5')
        
        # Load data safely
        try:
            with h5py.File(file_path, 'r') as h5f:
                image = h5f['image'][:]
                label = h5f['label'][:]
        except FileNotFoundError:
            raise FileNotFoundError(f"File not found: {file_path}")
        
        sample = {'image': image, 'label': label}
        if self.transform:
            sample = self.transform(sample)
        
        return sample


In [350]:
train_db = LAHeart(
    base_dir= '/kaggle/input/la-dataset/LA', 
    split= 'train'
)

sample =train_db.__getitem__(10)
print(f"Image.shape = {sample['image'].shape}")
print(f"Label.shape = {sample['label'].shape}")

Mode = train, total samples: 80
Image.shape = (203, 142, 88)
Label.shape = (203, 142, 88)


In [351]:
def random_rot_flip(image, label): 
    k = np.random.randint(0, 4, 1) 
    image = np.rot90(image, k) 
    label = np.rot90(label, k) 

    axis = np.random.randint(0, 2)
    image = np.flip(image, axis) 
    label = np.flip(label, axis) 

    return image, label 

class RandomRotFlip: 
    def __call__(self, sample): 
        image, label = sample['image'], sample['label']
        image, label = random_rot_flip(image, label) 
        sample = {'image': image, 'label': label}

        return sample 

In [352]:
class RandomCrop: 
    def __init__(self, output_size, with_sdf= False): 
        self.output_size = output_size 
        self.with_sdf = with_sdf
    
    def __call__(self, sample): 
        image, label = sample['image'], sample['label']

        if self.with_sdf: 
            sdf = sample['sdf']
        
        if label.shape[0] <= self.output_size[0] or label.shape[1] <= self.output_size[1] or label.shape[2] <= self.output_size[2]: 
            pw = max((self.output_size[0] - label.shape[0]) // 2 + 3, 0) 
            ph = max((self.output_size[1] - label.shape[1]) // 2 + 3, 0)
            pd = max((self.output_size[2] - label.shape[2]) // 2 + 3, 0)
            image = np.pad(image, [(pw, pw), (ph, ph),(pd, pd)], mode= 'constant', constant_values= 0)
            label = np.pad(label, [(pw, pw), (ph, ph), (pd, pd)], mode= 'constant', constant_values= 0) 

            if self.with_sdf: 
                sdf = np.pad(sdf, [(pw, pw), (ph, ph), (pd, pd)], mode= 'constant', constant_values= 0) 

        (w, h,d) = image.shape 
        w1 = np.random.randint(0, w - self.output_size[0])
        h1 = np.random.randint(0, h - self.output_size[1])
        d1 = np.random.randint(0, d - self.output_size[2])
    
        image = image[w1 : w1 + self.output_size[0], h1 : h1 + self.output_size[1], d1 : d1 + self.output_size[2]]
        label = label[w1: w1 + self.output_size[0], h1 : h1 + self.output_size[1], d1 : d1 + self.output_size[2]]

        if self.with_sdf: 
            sdf = sdf[w1 : w1 + self.output_size[0], h1 : h1 + self.output_size[1], d1 : d1 + self.output_size[2]]
            return {'image': image, 'label': label, 'sdf': sdf}
        else: 
            return {'image': image, 'label': label}


In [353]:
class ToTensor(object):
    def __call__(self, sample):
        image = sample['image'].copy()
        label = sample['label'].copy()
        
        # Ensure the image is reshaped correctly and avoid negative strides by copying the array
        image = image.reshape(1, image.shape[0], image.shape[1], image.shape[2]).astype(np.float32)
        
        if 'onehot_label' in sample:
            return {
                'image': torch.from_numpy(image),
                'label': torch.from_numpy(sample['label']).long(),
                'onehot_label': torch.from_numpy(sample['onehot_label']).long()
            }
        else:
            return {
                'image': torch.from_numpy(image),
                'label': torch.from_numpy(label).long()
            }


In [354]:
def iterate_once(indices): 
    """
    Permutate the iterable once 
    (permutate the labeled_idxs once)
    """
    return np.random.permutation(indices) 

def iterate_externally(indices): 
    """
    Create an infinite iterator that repeatedly permutes the indices.
    ( permutate the unlabeled_idxs to make different)
    """
    def infinite_shuffles(): 
        while True: 
            yield np.random.permutation(indices)
            
    return itertools.chain.from_iterable(infinite_shuffles())

def grouper(iterable, n): 
    args = [iter(iterable)] * n 
    return zip(*args)

In [355]:
class TwoStreamBatchSampler(Sampler):
    def __init__(self, primary_indices, secondary_indices, batch_size, secondary_batch_size):
        self.primary_indices = primary_indices
        self.secondary_indices = secondary_indices
        self.primary_batch_size = batch_size - secondary_batch_size
        self.secondary_batch_size = secondary_batch_size
        
        assert len(self.primary_indices) >= self.primary_batch_size > 0
        assert len(self.secondary_indices) >= self.secondary_batch_size > 0
        
    def __iter__(self):
        primary_iter = iterate_once(self.primary_indices)
        secondary_iter = iterate_externally(self.secondary_indices)
        return (
            primary_batch + secondary_batch
            for (primary_batch, secondary_batch)
            in zip(grouper(primary_iter, self.primary_batch_size),
                   grouper(secondary_iter, self.secondary_batch_size))
        )
        
    def __len__(self):
        return len(self.primary_indices) // self.primary_batch_size

In [356]:
def context_mask(img, mask_ratio):
    batch_size, channel, img_x, img_y, img_z = img.shape[0], img.shape[1], img.shape[2], img.shape[3], img.shape[4]
    loss_mask = torch.ones(batch_size, img_x, img_y, img_z).cuda()
    mask = torch.ones(img_x, img_y, img_z).cuda()
    patch_pixel_x, patch_pixel_y, patch_pixel_z = int(img_x*mask_ratio), int(img_y*mask_ratio), int(img_z*mask_ratio)
    w = np.random.randint(0, 112 - patch_pixel_x)
    h = np.random.randint(0, 112 - patch_pixel_y)
    z = np.random.randint(0, 80 - patch_pixel_z)
    mask[w:w+patch_pixel_x, h:h+patch_pixel_y, z:z+patch_pixel_z] = 0
    loss_mask[:, w:w+patch_pixel_x, h:h+patch_pixel_y, z:z+patch_pixel_z] = 0
    return mask.long(), loss_mask.long()

In [357]:
def test_single_case_plus(model_l, model_r, image, stride_xy, stride_z, patch_size, num_classes=1):
    w, h, d = image.shape

    # if the size of image is less than patch_size, then padding it
    add_pad = False
    if w < patch_size[0]:
        w_pad = patch_size[0]-w
        add_pad = True
    else:
        w_pad = 0
    if h < patch_size[1]:
        h_pad = patch_size[1]-h
        add_pad = True
    else:
        h_pad = 0
    if d < patch_size[2]:
        d_pad = patch_size[2]-d
        add_pad = True
    else:
        d_pad = 0
    wl_pad, wr_pad = w_pad//2,w_pad-w_pad//2
    hl_pad, hr_pad = h_pad//2,h_pad-h_pad//2
    dl_pad, dr_pad = d_pad//2,d_pad-d_pad//2
    if add_pad:
        image = np.pad(image, [(wl_pad,wr_pad),(hl_pad,hr_pad), (dl_pad, dr_pad)], mode='constant', constant_values=0)
    ww,hh,dd = image.shape

    sx = math.ceil((ww - patch_size[0]) / stride_xy) + 1
    sy = math.ceil((hh - patch_size[1]) / stride_xy) + 1
    sz = math.ceil((dd - patch_size[2]) / stride_z) + 1
    # print("{}, {}, {}".format(sx, sy, sz))
    score_map = np.zeros((num_classes, ) + image.shape).astype(np.float32)
    cnt = np.zeros(image.shape).astype(np.float32)

    for x in range(0, sx):
        xs = min(stride_xy*x, ww-patch_size[0])
        for y in range(0, sy):
            ys = min(stride_xy * y,hh-patch_size[1])
            for z in range(0, sz):
                zs = min(stride_z * z, dd-patch_size[2])
                test_patch = image[xs:xs+patch_size[0], ys:ys+patch_size[1], zs:zs+patch_size[2]]
                test_patch = np.expand_dims(np.expand_dims(test_patch,axis=0),axis=0).astype(np.float32)
                test_patch = torch.from_numpy(test_patch).cuda()

                with torch.no_grad():
                    y1_l, _ = model_l(test_patch)
                    y1_r, _ = model_r(test_patch)
                    y1 = (y1_l + y1_r) / 2
                    y = F.softmax(y1, dim=1)

                y = y.cpu().data.numpy()
                y = y[0,1,:,:,:]
                score_map[:, xs:xs+patch_size[0], ys:ys+patch_size[1], zs:zs+patch_size[2]] \
                  = score_map[:, xs:xs+patch_size[0], ys:ys+patch_size[1], zs:zs+patch_size[2]] + y
                cnt[xs:xs+patch_size[0], ys:ys+patch_size[1], zs:zs+patch_size[2]] \
                  = cnt[xs:xs+patch_size[0], ys:ys+patch_size[1], zs:zs+patch_size[2]] + 1
    score_map = score_map/np.expand_dims(cnt,axis=0)
    label_map = (score_map[0]>0.5).astype(np.int)
    if add_pad:
        label_map = label_map[wl_pad:wl_pad+w,hl_pad:hl_pad+h,dl_pad:dl_pad+d]
        score_map = score_map[:,wl_pad:wl_pad+w,hl_pad:hl_pad+h,dl_pad:dl_pad+d]
    return label_map, score_map

def var_all_case_LA_plus(model_l, model_r, num_classes, patch_size=(112, 112, 80), stride_xy=18, stride_z=4):
   
    with open('/data/byh_data/SSNet_data/LA/test.list', 'r') as f:
        image_list = f.readlines()
    image_list = ["/data/byh_data/SSNet_data/LA/2018LA_Seg_Training Set/" + item.replace('\n', '') + "/mri_norm2.h5" for item in image_list]
    loader = tqdm(image_list)
    total_dice = 0.0
    for image_path in loader:
        h5f = h5py.File(image_path, 'r')
        image = h5f['image'][:]
        label = h5f['label'][:]
        prediction, score_map = test_single_case_plus(model_l, model_r, image, stride_xy, stride_z, patch_size, num_classes=num_classes)
        if np.sum(prediction)==0:
            dice = 0
        else:
            dice = metric.binary.dc(prediction, label)
        total_dice += dice
    avg_dice = total_dice / len(image_list)
    print('average metric is {}'.format(avg_dice))
    return avg_dice

#### 2.Loss

In [358]:
def to_one_hot(tensor, nclasses): 
    """
    Input (tensor): Nx1xHxW 
    """
    assert tensor.max().item() < nclasses
    assert tensor.min().item() >= 0 

    size = list(tensor.size())
    assert size[1] == 1 
    size[1] = nclasses
    one_hot = torch.zeros(*size) 
    if tensor.is_cuda: 
        one_hot = one_hot.cuda(tensor.device) 
    one_hot = one_hot.scatter_(1, tensor, 1) 
    return one_hot 

def get_probability(logits): 
    """
    Get the probability from logitis  
    """
    size = logits.size() 
    if size[1] > 1: 
        pred = F.softmax(logits, dim= 1) 
        nclass = size[1] 
    else: 
        pred = F.sigmoid(logits) 
        pred = torch.cat([1 - pred, pred], dim= 1) 
    
    return pred, nclass


class mask_DiceLoss(nn.Module): 
    def __init__(self, nclass, class_weights = None, smooth= 1e-5): 
        super(mask_DiceLoss, self).__init__() 
        self.smooth = smooth 
        if class_weights is None: 
            self.class_weights = nn.Parameter(torch.ones((1, nclass)).type(torch.float32), requires_grad= False) 
        else: 
            class_weights = np.array(class_weights) 
            assert nclass == class_weights.shape[0] 
            self.class_weights = nn.Parameter(torch.tensor(class_weights, dtype= torch.float32), requires_grad= False) 
    
    def prob_forward(self, pred, target, mask= None): 
        size = pred.size() 
        N, nclass = size[0], size[1] 

        # N x C x H x W: convert into 2D image
        pred_one_hot = pred.view(N, nclass, -1) 
        target = target.view(N, 1, -1) 
        target_one_hot = to_one_hot(target.type(torch.long), nclass).type(torch.float32)

        # N x C x H x W
        inter = pred_one_hot * target_one_hot
        union = pred_one_hot + target_one_hot

        if mask is not None: 
            mask = mask.view(N, 1, -1) 
            inter = (inter.view(N, nclass, -1) * mask).sum(2) 
            union = (union.view(N, nclass, -1) * mask).sum(2) 
        else: 
            inter = inter.view(N, nclass, -1).sum(2) 
            union = union.view(N, nclass, -1).sum(2)
        
        dice = ( 2*inter + self.smooth ) / (union + self.smooth) 
        return 1 - dice.mean()

    def forward(self, logits,target, mask = None): 
        size = logits.size() 
        N, nclass = size[0], size[1] 

        logits = logits.view(N, nclass, -1) 
        target = target.view(N, 1, -1) 

        pred, nclass = get_probability(logits) 

        pred_one_hot = pred 
        target_one_hot = to_one_hot(target.type(torch.long), nclass).type(torch.float32) 

        inter = pred_one_hot * target_one_hot
        union = pred_one_hot + target_one_hot

        if mask is not None: 
            mask = mask.view(N, 1, -1) 
            inter = (inter.view(N, nclass, -1) * mask).sum(2)
            union = (union.view(N, nclass, -1) * mask ).sum(2) 
        else: 
            inter = inter.view(N, nclass, -1).sum(2) 
            union = union.view(N, nclass, -1).sum(2)
        
        dice = ( 2 * inter + self.smooth ) / (union + self.smooth)
        return 1 - dice.mean() 
        


#### VNet

In [359]:
import torch
from torch import nn
import pdb


class ConvBlock(nn.Module):
    def __init__(self, n_stages, n_filters_in, n_filters_out, kernel_size=3, padding=1, normalization='none'):
        super(ConvBlock, self).__init__()

        ops = []
        for i in range(n_stages):
            if i==0:
                input_channel = n_filters_in
            else:
                input_channel = n_filters_out

            ops.append(nn.Conv3d(input_channel, n_filters_out, kernel_size=kernel_size, padding=padding))
            if normalization == 'batchnorm':
                ops.append(nn.BatchNorm3d(n_filters_out))
            elif normalization == 'groupnorm':
                ops.append(nn.GroupNorm(num_groups=16, num_channels=n_filters_out))
            elif normalization == 'instancenorm':
                ops.append(nn.InstanceNorm3d(n_filters_out))
            elif normalization != 'none':
                assert False
            ops.append(nn.ReLU(inplace=True))

        self.conv = nn.Sequential(*ops)

    def forward(self, x):
        x = self.conv(x)
        return x


class ResidualConvBlock(nn.Module):
    def __init__(self, n_stages, n_filters_in, n_filters_out, normalization='none'):
        super(ResidualConvBlock, self).__init__()

        ops = []
        for i in range(n_stages):
            if i == 0:
                input_channel = n_filters_in
            else:
                input_channel = n_filters_out

            ops.append(nn.Conv3d(input_channel, n_filters_out, 3, padding=1))
            if normalization == 'batchnorm':
                ops.append(nn.BatchNorm3d(n_filters_out))
            elif normalization == 'groupnorm':
                ops.append(nn.GroupNorm(num_groups=16, num_channels=n_filters_out))
            elif normalization == 'instancenorm':
                ops.append(nn.InstanceNorm3d(n_filters_out))
            elif normalization != 'none':
                assert False

            if i != n_stages-1:
                ops.append(nn.ReLU(inplace=True))

        self.conv = nn.Sequential(*ops)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = (self.conv(x) + x)
        x = self.relu(x)
        return x


class DownsamplingConvBlock(nn.Module):
    def __init__(self, n_filters_in, n_filters_out, stride=2, padding=0, normalization='none'):
        super(DownsamplingConvBlock, self).__init__()

        ops = []
        if normalization != 'none':
            ops.append(nn.Conv3d(n_filters_in, n_filters_out, stride, padding=padding, stride=stride))
            if normalization == 'batchnorm':
                ops.append(nn.BatchNorm3d(n_filters_out))
            elif normalization == 'groupnorm':
                ops.append(nn.GroupNorm(num_groups=16, num_channels=n_filters_out))
            elif normalization == 'instancenorm':
                ops.append(nn.InstanceNorm3d(n_filters_out))
            else:
                assert False
        else:
            ops.append(nn.Conv3d(n_filters_in, n_filters_out, stride, padding=padding, stride=stride))

        ops.append(nn.ReLU(inplace=True))

        self.conv = nn.Sequential(*ops)

    def forward(self, x):
        x = self.conv(x)
        return x


class UpsamplingDeconvBlock(nn.Module):
    def __init__(self, n_filters_in, n_filters_out, stride=2, padding=0,normalization='none'):
        super(UpsamplingDeconvBlock, self).__init__()

        ops = []
        if normalization != 'none':
            ops.append(nn.ConvTranspose3d(n_filters_in, n_filters_out, stride, padding=padding, stride=stride))
            if normalization == 'batchnorm':
                ops.append(nn.BatchNorm3d(n_filters_out))
            elif normalization == 'groupnorm':
                ops.append(nn.GroupNorm(num_groups=16, num_channels=n_filters_out))
            elif normalization == 'instancenorm':
                ops.append(nn.InstanceNorm3d(n_filters_out))
            else:
                assert False
        else:
            ops.append(nn.ConvTranspose3d(n_filters_in, n_filters_out, stride, padding=padding, stride=stride))

        ops.append(nn.ReLU(inplace=True))

        self.conv = nn.Sequential(*ops)

    def forward(self, x):
        x = self.conv(x)
        return x
    

class Upsampling(nn.Module):
    def __init__(self, n_filters_in, n_filters_out, stride=2, normalization='none'):
        super(Upsampling, self).__init__()

        ops = []
        ops.append(nn.Upsample(scale_factor=stride, mode="trilinear",align_corners=False))
        ops.append(nn.Conv3d(n_filters_in, n_filters_out, kernel_size=3, padding=1))
        if normalization == 'batchnorm':
            ops.append(nn.BatchNorm3d(n_filters_out))
        elif normalization == 'groupnorm':
            ops.append(nn.GroupNorm(num_groups=16, num_channels=n_filters_out))
        elif normalization == 'instancenorm':
            ops.append(nn.InstanceNorm3d(n_filters_out))
        elif normalization != 'none':
            assert False
        ops.append(nn.ReLU(inplace=True))

        self.conv = nn.Sequential(*ops)

    def forward(self, x):
        x = self.conv(x)
        return x
    
class Encoder(nn.Module):
    def __init__(self, n_channels=3, n_classes=2, n_filters=16, normalization='none', has_dropout=False, has_residual=False):
        super(Encoder, self).__init__()
        self.has_dropout = has_dropout
        convBlock = ConvBlock if not has_residual else ResidualConvBlock

        self.block_one = convBlock(1, n_channels, n_filters, normalization=normalization)
        self.block_one_dw = DownsamplingConvBlock(n_filters, 2 * n_filters, normalization=normalization)

        self.block_two = convBlock(2, n_filters * 2, n_filters * 2, normalization=normalization)
        self.block_two_dw = DownsamplingConvBlock(n_filters * 2, n_filters * 4, normalization=normalization)

        self.block_three = convBlock(3, n_filters * 4, n_filters * 4, normalization=normalization)
        self.block_three_dw = DownsamplingConvBlock(n_filters * 4, n_filters * 8, normalization=normalization)

        self.block_four = convBlock(3, n_filters * 8, n_filters * 8, normalization=normalization)
        self.block_four_dw = DownsamplingConvBlock(n_filters * 8, n_filters * 16, normalization=normalization)

        self.block_five = convBlock(3, n_filters * 16, n_filters * 16, normalization=normalization)
        
        self.dropout = nn.Dropout3d(p=0.5, inplace=False)

    def forward(self, input):
        x1 = self.block_one(input)
        x1_dw = self.block_one_dw(x1)

        x2 = self.block_two(x1_dw)
        x2_dw = self.block_two_dw(x2)

        x3 = self.block_three(x2_dw)
        x3_dw = self.block_three_dw(x3)

        x4 = self.block_four(x3_dw)
        x4_dw = self.block_four_dw(x4)

        x5 = self.block_five(x4_dw)

        if self.has_dropout:
            x5 = self.dropout(x5)

        res = [x1, x2, x3, x4, x5]
        return res


class Decoder(nn.Module):
    def __init__(self, n_channels=3, n_classes=2, n_filters=16, normalization='none', has_dropout=False, has_residual=False):
        super(Decoder, self).__init__()
        self.has_dropout = has_dropout

        convBlock = ConvBlock if not has_residual else ResidualConvBlock

        upsampling = UpsamplingDeconvBlock ## using transposed convolution

        self.block_five_up = upsampling(n_filters * 16, n_filters * 8, normalization=normalization)

        self.block_six = convBlock(3, n_filters * 8, n_filters * 8, normalization=normalization)
        self.block_six_up = upsampling(n_filters * 8, n_filters * 4, normalization=normalization)

        self.block_seven = convBlock(3, n_filters * 4, n_filters * 4, normalization=normalization)
        self.block_seven_up = upsampling(n_filters * 4, n_filters * 2, normalization=normalization)

        self.block_eight = convBlock(2, n_filters * 2, n_filters * 2, normalization=normalization)
        self.block_eight_up = upsampling(n_filters * 2, n_filters, normalization=normalization)

        self.block_nine = convBlock(1, n_filters, n_filters, normalization=normalization)
        self.out_conv = nn.Conv3d(n_filters, n_classes, 1, padding=0)
        self.dropout = nn.Dropout3d(p=0.5, inplace=False)

    def forward(self, features):
        x1 = features[0]
        x2 = features[1]
        x3 = features[2]
        x4 = features[3]
        x5 = features[4]
        
        x5_up = self.block_five_up(x5)
        x5_up = x5_up + x4

        x6 = self.block_six(x5_up)
        x6_up = self.block_six_up(x6)
        x6_up = x6_up + x3

        x7 = self.block_seven(x6_up)
        x7_up = self.block_seven_up(x7)
        x7_up = x7_up + x2

        x8 = self.block_eight(x7_up)
        x8_up = self.block_eight_up(x8)
        x8_up = x8_up + x1
        x9 = self.block_nine(x8_up)
        # x9 = F.dropout3d(x9, p=0.5, training=True)
        if self.has_dropout:
            x9 = self.dropout(x9)
        out_seg = self.out_conv(x9)
        return out_seg, x8_up
 
class VNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=2, n_filters=16, normalization='none', has_dropout=False, has_residual=False):
        super(VNet, self).__init__()

        self.encoder = Encoder(n_channels, n_classes, n_filters, normalization, has_dropout, has_residual)
        self.decoder = Decoder(n_channels, n_classes, n_filters, normalization, has_dropout, has_residual)
        dim_in = 16
        feat_dim = 32
        self.pool = nn.MaxPool3d(3, stride=2)
        self.projection_head = nn.Sequential(
            nn.Linear(dim_in, feat_dim),
            nn.BatchNorm1d(feat_dim),
            nn.ReLU(inplace=True),
            nn.Linear(feat_dim, feat_dim)
        )
        self.prediction_head = nn.Sequential(
            nn.Linear(feat_dim, feat_dim),
            nn.BatchNorm1d(feat_dim),
            nn.ReLU(inplace=True),
            nn.Linear(feat_dim, feat_dim)
        )
        for class_c in range(2):
            selector = nn.Sequential(
                nn.Linear(feat_dim, feat_dim),
                nn.BatchNorm1d(feat_dim),
                nn.LeakyReLU(negative_slope=0.2, inplace=True),
                nn.Linear(feat_dim, 1)
            )
            self.__setattr__('contrastive_class_selector_' + str(class_c), selector)

        for class_c in range(2):
            selector = nn.Sequential(
                nn.Linear(feat_dim, feat_dim),
                nn.BatchNorm1d(feat_dim),
                nn.LeakyReLU(negative_slope=0.2, inplace=True),
                nn.Linear(feat_dim, 1)
            )
            self.__setattr__('contrastive_class_selector_memory' + str(class_c), selector)
        
    def forward_projection_head(self, features):
        return self.projection_head(features)

    def forward_prediction_head(self, features):
        return self.prediction_head(features)

    def forward(self, input):
        features = self.encoder(input)
        out_seg, x8_up = self.decoder(features)
        features = self.pool(features[4])
        return out_seg, features # 4, 16, 112, 112, 80

In [360]:
def sigmoid_rampup(current, rampup_length):
    if rampup_length == 0: 
        return 1.0 
    else:
        current = np.clip(current, 0, rampup_length)
        phase = 1 - (current / rampup_length)
        return float(np.exp(-5 * phase * phase))
    
# Mean-Teacher compomnent 
def get_current_consistency_weight(epoch, args): 
    return 5 * args.consistency + sigmoid_rampup(epoch, args.consistency_rampup)

def update_model_ema(model, ema_model, alpha): 
    model_state = model.state_dict() 
    model_ema_state = ema_model.state_dict()


    new_dict = {} 

    for key in model_state:
        new_dict[key] = alpha * model_ema_state[key] + (1 - alpha) * model_state[key]

    ema_model.load_state_dict(new_dict)

In [361]:
def get_cut_mask(out, thres=0.5, nms=0):
    probs = F.softmax(out, 1)
    masks = (probs >= thres).type(torch.int64)
    masks = masks[:, 1, :, :].contiguous()
    if nms == 1:
        masks = LargestCC_pancreas(masks)
    return masks

def LargestCC_pancreas(segmentation):
    N = segmentation.shape[0]
    batch_list = []
    for n in range(N):
        n_prob = segmentation[n].detach().cpu().numpy()
        labels = label(n_prob)
        if labels.max() != 0:
            largestCC = labels == np.argmax(np.bincount(labels.flat)[1:])+1
        else:
            largestCC = n_prob
        batch_list.append(largestCC)
    
    return torch.Tensor(batch_list).cuda()

def save_net_opt(net, optimizer, path):
    state = {
        'net': net.state_dict(),
        'opt': optimizer.state_dict(),
    }
    torch.save(state, str(path))

def load_net_opt(net, optimizer, path):
    state = torch.load(str(path))
    net.load_state_dict(state['net'])
    optimizer.load_state_dict(state['opt'])

def load_net(net, path):
    state = torch.load(str(path))
    net.load_state_dict(state['net'])

def get_current_consistency_weight(epoch):
    return args.consistency * sigmoid_rampup(epoch, args.consistency_rampup)

In [362]:
def net_factory(net_type="Unet", in_channels=1, class_num=2, mode="train", tsne=0):
    if net_type == "VNet" and mode == "train" and tsne==0:
        net = VNet(n_channels=in_channels, n_classes=class_num, normalization='batchnorm', has_dropout=True).cuda()
        
    return net

In [363]:
train_data_path = args.root_path 
pre_max_iterations = args.pre_max_iterations
self_max_iterations = args.self_max_iteration 
base_lr = args.base_lr 
CE = nn.CrossEntropyLoss(reduction= 'none')


if args.deterministic:
    cudnn.benchmark = False
    cudnn.deterministic = True
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    random.seed(args.seed)
    np.random.seed(args.seed)

patch_size = (112, 112, 80)
num_classes = 2

In [364]:
def pre_train(args, snapshot_path):
    model = net_factory(args.model, in_channels=1, class_num=num_classes, mode="train")
    db_train = LAHeart(base_dir=train_data_path, 
                       split='train',
                       transform=transforms.Compose([
                           RandomRotFlip(),
                           RandomCrop(patch_size),
                           ToTensor(),
                       ]))
    labelnum = args.labelnum
    labeled_idxs = list(range(labelnum))
    unlabeled_idxs = list(range(labelnum, args.max_samples))
    batch_sampler = TwoStreamBatchSampler(labeled_idxs, unlabeled_idxs, args.batch_size, args.batch_size-args.labeled_bs)
    sub_bs = int(args.labeled_bs/2)
    def worker_init_fn(worker_id):
        random.seed(args.seed+worker_id)
    trainloader = DataLoader(db_train, batch_sampler=batch_sampler, num_workers=4, pin_memory=True, worker_init_fn=worker_init_fn)
    optimizer = SGD(model.parameters(), lr=base_lr, momentum=0.9, weight_decay=1e-4)
    DICE = mask_DiceLoss(nclass=2)
    
    model.train()
    writer = SummaryWriter(snapshot_path + '/log')
    logging.info("{} iterations per epoch".format(len(trainloader)))
    iter_num = 0
    best_dice = 0
    max_epoch = pre_max_iterations // len(trainloader) + 1
    iterator = tqdm(range(max_epoch), ncols=70)
    for epoch_num in iterator:
        for _, sampled_batch in enumerate(trainloader):
            volume_batch, label_batch = sampled_batch['image'][:args.labeled_bs], sampled_batch['label'][:args.labeled_bs]
            volume_batch, label_batch = volume_batch.cuda(), label_batch.cuda()
            img_a, img_b = volume_batch[:sub_bs], volume_batch[sub_bs:]
            lab_a, lab_b = label_batch[:sub_bs], label_batch[sub_bs:]
            
            with torch.no_grad():
                img_mask, loss_mask = context_mask(img_a, args.mask_ratio)
                
            # Mix Input
            volume_batch = img_a * img_mask + img_b * (1 - img_mask)
            label_batch = lab_a * img_mask + lab_b * (1 - img_mask)
            
            outputs, _ = model(volume_batch)
            loss_ce = F.cross_entropy(outputs, label_batch)
            loss_dice = DICE(outputs, label_batch)
            loss = (loss_ce + loss_dice) / 2
            
            iter_num += 1
            writer.add_scalar('pre/loss_dice', loss_dice, iter_num)
            writer.add_scalar('pre/loss_ce', loss_ce, iter_num)
            writer.add_scalar('pre/loss_all', loss, iter_num)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            logging.info('iteration %d : loss: %03f, loss_dice: %03f, loss_ce: %03f' %(iter_num, loss, loss_dice, loss_ce))
            
            if iter_num >= pre_max_iterations:
                break

        if iter_num >= pre_max_iterations:
            iterator.close()
            break
    writer.close()

In [365]:
# Params 
class params: 
    def __init__(self):
        self.root_path = '/kaggle/input/la-dataset/LA'
        self.exp = 'BCP' 
        self.model = 'VNet'
        self.pre_max_iterations = 200
        self.self_max_iteration = 100 
        self.max_samples = 80 
        self.labeled_bs = 4
        self.batch_size = 8 
        self.base_lr = 0.01 
        self.deterministic = 1 
        self.labelnum = 8 
        self.consistency = 1.0 
        self.consistency_rampup = 40.0 
        self.magnitude = 10.0 
        self.seed = 10
    
        # Setting of BCP 
        self.u_weight = 0.5 
        self.mask_ratio = 2/3 

        # Setting of mixup 
        self.u_alpha = 2.0 
        self.loss_weight = 0.5


args = params()

In [366]:
 ## make logger file
pre_snapshot_path = "./model/BCP/LA_{}_{}_labeled/pre_train".format(args.exp, args.labelnum)
self_snapshot_path = "./model/BCP/LA_{}_{}_labeled/self_train".format(args.exp, args.labelnum)
print("Starting BCP training.")
for snapshot_path in [pre_snapshot_path, self_snapshot_path]:
    if not os.path.exists(snapshot_path):
        os.makedirs(snapshot_path)
# -- Pre-Training
logging.basicConfig(filename=pre_snapshot_path+"/log.txt", level=logging.INFO, format='[%(asctime)s.%(msecs)03d] %(message)s', datefmt='%H:%M:%S')
logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))
logging.info(str(args))
pre_train(args, pre_snapshot_path)

Starting BCP training.
Mode = train, total samples: 80
2 iterations per epoch
2 iterations per epoch
2 iterations per epoch
2 iterations per epoch
2 iterations per epoch
2 iterations per epoch
2 iterations per epoch
2 iterations per epoch
2 iterations per epoch
2 iterations per epoch
2 iterations per epoch
2 iterations per epoch
2 iterations per epoch
2 iterations per epoch
2 iterations per epoch
2 iterations per epoch
2 iterations per epoch
2 iterations per epoch
2 iterations per epoch
2 iterations per epoch
2 iterations per epoch


  0%|                                         | 0/101 [00:00<?, ?it/s]

iteration 1 : loss: 0.680336, loss_dice: 0.582472, loss_ce: 0.778200
iteration 1 : loss: 0.680336, loss_dice: 0.582472, loss_ce: 0.778200
iteration 1 : loss: 0.680336, loss_dice: 0.582472, loss_ce: 0.778200
iteration 1 : loss: 0.680336, loss_dice: 0.582472, loss_ce: 0.778200
iteration 1 : loss: 0.680336, loss_dice: 0.582472, loss_ce: 0.778200
iteration 1 : loss: 0.680336, loss_dice: 0.582472, loss_ce: 0.778200
iteration 1 : loss: 0.680336, loss_dice: 0.582472, loss_ce: 0.778200
iteration 1 : loss: 0.680336, loss_dice: 0.582472, loss_ce: 0.778200
iteration 1 : loss: 0.680336, loss_dice: 0.582472, loss_ce: 0.778200
iteration 1 : loss: 0.680336, loss_dice: 0.582472, loss_ce: 0.778200
iteration 1 : loss: 0.680336, loss_dice: 0.582472, loss_ce: 0.778200
iteration 1 : loss: 0.680336, loss_dice: 0.582472, loss_ce: 0.778200
iteration 1 : loss: 0.680336, loss_dice: 0.582472, loss_ce: 0.778200
iteration 1 : loss: 0.680336, loss_dice: 0.582472, loss_ce: 0.778200
iteration 1 : loss: 0.680336, loss

  1%|▎                                | 1/101 [00:03<05:16,  3.16s/it]

iteration 3 : loss: 0.767540, loss_dice: 0.612033, loss_ce: 0.923048
iteration 3 : loss: 0.767540, loss_dice: 0.612033, loss_ce: 0.923048
iteration 3 : loss: 0.767540, loss_dice: 0.612033, loss_ce: 0.923048
iteration 3 : loss: 0.767540, loss_dice: 0.612033, loss_ce: 0.923048
iteration 3 : loss: 0.767540, loss_dice: 0.612033, loss_ce: 0.923048
iteration 3 : loss: 0.767540, loss_dice: 0.612033, loss_ce: 0.923048
iteration 3 : loss: 0.767540, loss_dice: 0.612033, loss_ce: 0.923048
iteration 3 : loss: 0.767540, loss_dice: 0.612033, loss_ce: 0.923048
iteration 3 : loss: 0.767540, loss_dice: 0.612033, loss_ce: 0.923048
iteration 3 : loss: 0.767540, loss_dice: 0.612033, loss_ce: 0.923048
iteration 3 : loss: 0.767540, loss_dice: 0.612033, loss_ce: 0.923048
iteration 3 : loss: 0.767540, loss_dice: 0.612033, loss_ce: 0.923048
iteration 3 : loss: 0.767540, loss_dice: 0.612033, loss_ce: 0.923048
iteration 3 : loss: 0.767540, loss_dice: 0.612033, loss_ce: 0.923048
iteration 3 : loss: 0.767540, loss

...

 97%|███████████████████████████████ | 98/101 [03:20<00:06,  2.07s/it]

iteration 197 : loss: 0.234730, loss_dice: 0.258442, loss_ce: 0.211018
iteration 197 : loss: 0.234730, loss_dice: 0.258442, loss_ce: 0.211018
iteration 197 : loss: 0.234730, loss_dice: 0.258442, loss_ce: 0.211018
iteration 197 : loss: 0.234730, loss_dice: 0.258442, loss_ce: 0.211018
iteration 197 : loss: 0.234730, loss_dice: 0.258442, loss_ce: 0.211018
iteration 197 : loss: 0.234730, loss_dice: 0.258442, loss_ce: 0.211018
iteration 197 : loss: 0.234730, loss_dice: 0.258442, loss_ce: 0.211018
iteration 197 : loss: 0.234730, loss_dice: 0.258442, loss_ce: 0.211018
iteration 197 : loss: 0.234730, loss_dice: 0.258442, loss_ce: 0.211018
iteration 197 : loss: 0.234730, loss_dice: 0.258442, loss_ce: 0.211018
iteration 197 : loss: 0.234730, loss_dice: 0.258442, loss_ce: 0.211018
iteration 197 : loss: 0.234730, loss_dice: 0.258442, loss_ce: 0.211018
iteration 197 : loss: 0.234730, loss_dice: 0.258442, loss_ce: 0.211018
iteration 197 : loss: 0.234730, loss_dice: 0.258442, loss_ce: 0.211018
iterat

 98%|███████████████████████████████▎| 99/101 [03:22<00:04,  2.06s/it]

iteration 199 : loss: 0.242147, loss_dice: 0.291255, loss_ce: 0.193038
iteration 199 : loss: 0.242147, loss_dice: 0.291255, loss_ce: 0.193038
iteration 199 : loss: 0.242147, loss_dice: 0.291255, loss_ce: 0.193038
iteration 199 : loss: 0.242147, loss_dice: 0.291255, loss_ce: 0.193038
iteration 199 : loss: 0.242147, loss_dice: 0.291255, loss_ce: 0.193038
iteration 199 : loss: 0.242147, loss_dice: 0.291255, loss_ce: 0.193038
iteration 199 : loss: 0.242147, loss_dice: 0.291255, loss_ce: 0.193038
iteration 199 : loss: 0.242147, loss_dice: 0.291255, loss_ce: 0.193038
iteration 199 : loss: 0.242147, loss_dice: 0.291255, loss_ce: 0.193038
iteration 199 : loss: 0.242147, loss_dice: 0.291255, loss_ce: 0.193038
iteration 199 : loss: 0.242147, loss_dice: 0.291255, loss_ce: 0.193038
iteration 199 : loss: 0.242147, loss_dice: 0.291255, loss_ce: 0.193038
iteration 199 : loss: 0.242147, loss_dice: 0.291255, loss_ce: 0.193038
iteration 199 : loss: 0.242147, loss_dice: 0.291255, loss_ce: 0.193038
iterat

 98%|███████████████████████████████▎| 99/101 [03:25<00:04,  2.07s/it]
